# Create ES

In [ ]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
from elasticsearch.client import IndicesClient
from requests_aws4auth import AWS4Auth
import boto3


es = Elasticsearch([{'host': 'localhost', 'port': '9200'}])    
IndicesClient(es).put_settings(body={
      "index.mapping.total_fields.limit": 1000
    }, index=index)

#### or AWS...

In [ ]:
host = 'search-############################.ap-southeast-1.es.amazonaws.com'
region = 'ap-southeast-1'
service = 'es'
credentials = boto3.Session(aws_access_key_id=#aws_access_key_id#,
                            aws_secret_access_key=#aws_secret_access_key#,
                            ).get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service)

es = Elasticsearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = awsauth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

---
### 1. Normal Insert

In [ ]:
document = {
    "title": "Moneyball",
    "director": "Bennett Miller",
    "year": "2011"
}
es.index(index="movies", doc_type="_doc", id="5", body=document)
print(es.get(index="movies", doc_type="_doc", id="5"))

In [ ]:
es.indices.delete(index='movies', ignore=[400, 404])

### 2. Bulkload Helpers

In [ ]:
from elasticsearch import helpers
import csv

with open("test.csv") as file:
    reader = csv.DictReader(file)
    helpers.bulk(es, reader, index="TEST".lower(), doc_type="_doc", timeout=30)

### 3. Dataframe for loop Insertion

In [2]:
import pandas as pd

df = pd.DataFrame([['Harry Potter',2010],['Inception',2018]], columns=['movie','year'])
df

,movie,year
0,Harry Potter,2010
1,Inception,2018


In [ ]:
import json

for i,j in enumerate(df.to_dict(orient='records')):
    df_json = json.dumps(j)
    try:
        es.index(index='movies', doc_type='_doc', id=i, body=df_json) #ignore=400, 
    except Exception as e:
        print(str(e))
        print(df_json)
        break